# Q Business by API
## Creating an Amazon Q Business instance, a Box Datasource, and debugging the datasource by JSON configuration

This notebook provides a comprehensive guide to programmatically setting up and managing Amazon Q Business with Box data sources using the AWS SDK for Python (boto3). Unlike the console experience, this API-driven approach gives you greater control and automation capabilities for your Q Business deployments.

What you'll learn:
- How to create an Amazon Q Business instance with proper IAM roles and permissions
- Setting up a Box data source with customized configuration
- Adding advanced configurations through JSON configuration
- Monitoring and troubleshooting data source synchronization
- Working with indexes and retrievers for effective document search
- Managing AWS session tokens for long-running operations

Why use this approach?

The API-driven approach demonstrated in this notebook offers several advantages:
- Automation: Easily replicate your Q Business setup across multiple environments
- Advanced Configuration: Access configuration options through JSON configuration
- Troubleshooting: Gain deeper insights into synchronization issues and Box connector behavior
- Comparison: Export configurations to compare different instances and identify discrepancies
- Support: Generate detailed configuration exports to share with AWS Support for troubleshooting

This notebook serves as both a practical implementation guide and a reference for common debugging scenarios when working with Amazon Q Business Box connectors.




## Prerequisites

1. AWS CLI configured with appropriate credentials
2. Required Python packages installed
3. Appropriate IAM permissions to create Q Business instances and IAM roles

In [ ]:
import boto3
import json
import botocore
import time
import random
import string
import os
import ssl
import urllib3
from datetime import datetime

## Define some variables

Set the values to what's applicable to your configuration


In [ ]:
# Generate a random 5-character suffix
def generate_suffix(length=5):
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


# AWS Region
region = "us-west-2"  # Replace with your desired region

# Role names prefix
box_role_name = "QBusiness-Box-DataSource-"

# Application name prefix
instance_name = "my-q-business-application-"

# Box Secret prefix
box_secret_name = "QBusiness-Box-"

# Add random suffix to names
suffix = generate_suffix()
box_role_name = f"{box_role_name}{suffix}"
instance_name = f"{instance_name}{suffix}"
box_secret_name = f"{box_secret_name}{suffix}"

# User configuration
email = "john.doe@example.com"
first_name = "John"
family_name = "Doe"

## Show info about our instance

In [ ]:
print(f"Using Box role name: {box_role_name}")
print(f"Using instance name: {instance_name}")
print(f"Using secret name: {box_secret_name}")

## Box Configuration

In [ ]:
# Box configuration
client_id = "your clientid"
client_secret = "your secret"
public_key_id = "your publickeyid"
private_key = "your private key"
passphrase = "your passphrase"

box_enterprise_id = "your enterprise id"

## Validate AWS credentials

Verify credentials. If not, configure them before continuing this notebook.


In [ ]:
# Disable SSL warnings and verification
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
os.environ["PYTHONHTTPSVERIFY"] = "0"
ssl._create_default_https_context = ssl._create_unverified_context

# Check if AWS credentials are properly configured
try:
    # Try to get caller identity (lightweight AWS API call)
    sts = boto3.client("sts", region_name=region, verify=False)
    identity = sts.get_caller_identity()

    print("✅ AWS credentials are properly configured!")
    print(f"Account ID: {identity['Account']}")
    print(f"User ID: {identity['UserId']}")
    print(f"ARN: {identity['Arn']}")
except Exception as e:
    print("❌ AWS credentials are not properly configured!")
    print(f"Error: {str(e)}")

## Create a secret for box credentials

In [ ]:
# Create client with SSL verification disabled
secrets_client = boto3.client("secretsmanager", region_name=region, verify=False)

box_credentials = {
    "clientID": client_id,  # Updated variable name
    "clientSecret": client_secret,  # Updated variable name
    "publicKeyID": public_key_id,  # Updated variable name
    "privateKey": private_key,  # Updated variable name
    "passphrase": passphrase,
}

try:
    response = secrets_client.create_secret(
        Name=box_secret_name, SecretString=json.dumps(box_credentials)
    )
    print(f"Secret created: {response['ARN']}")
except secrets_client.exceptions.ResourceExistsException:
    print(f"Secret '{box_secret_name}' already exists")

## Create IAM Role for Q Business

In most accounts, this AWS managed role will already exist. This cell creates an IAM role with a trust policy for Q Business. The role will be used by the Q Business instance to access AWS resources.

In [ ]:
# Refresh session to ensure credentials are valid
session = boto3.Session(region_name=region)
iam = session.client("iam", region_name=region, verify=False)

# Use the managed service-linked role for Amazon Q Business application
app_role_arn = (
    f"arn:aws:iam::{identity['Account']}:"
    f"role/aws-service-role/qbusiness.amazonaws.com/AWSServiceRoleForQBusiness"
)

# Check if the service-linked role exists, create it if it doesn't
try:
    iam.get_role(RoleName="AWSServiceRoleForQBusiness")
    print("✅ Service-linked role for Q Business already exists")
except iam.exceptions.NoSuchEntityException:
    print("Creating service-linked role for Q Business...")
    iam.create_service_linked_role(AWSServiceName="qbusiness.amazonaws.com")
    print("✅ Service-linked role for Q Business created")

print(f"Using service-linked role for application: {app_role_arn}")

## Create a role for the Box datasource

Best practice is to have a separate role for each data source.

In [ ]:
# Create a separate role for the Box datasource with specific permissions
box_trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {"Service": "qbusiness.amazonaws.com"},
            "Action": "sts:AssumeRole",
        }
    ],
}

In [ ]:
# Define custom policy for the Box datasource
box_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowsAmazonQToGetS3Objects",
            "Action": ["s3:GetObject"],
            "Resource": ["arn:aws:s3:::*/*"],
            "Effect": "Allow",
            "Condition": {"StringEquals": {"aws:ResourceAccount": identity["Account"]}},
        },
        {
            "Sid": "AllowsAmazonQToGetSecret",
            "Effect": "Allow",
            "Action": ["secretsmanager:GetSecretValue"],
            "Resource": [
                f"arn:aws:secretsmanager:{region}:{identity['Account']}:secret:*"
            ],
        },
        {
            "Sid": "AllowsAmazonQToDecryptSecret",
            "Effect": "Allow",
            "Action": ["kms:Decrypt"],
            "Resource": [f"arn:aws:kms:{region}:{identity['Account']}:key/*"],
            "Condition": {
                "StringLike": {"kms:ViaService": ["secretsmanager.*.amazonaws.com"]}
            },
        },
        {
            "Sid": "AllowsAmazonQToIngestDocuments",
            "Effect": "Allow",
            "Action": ["qbusiness:BatchPutDocument", "qbusiness:BatchDeleteDocument"],
            "Resource": "*",
        },
        {
            "Sid": "AllowsAmazonQToIngestPrincipalMapping",
            "Effect": "Allow",
            "Action": [
                "qbusiness:PutGroup",
                "qbusiness:CreateUser",
                "qbusiness:DeleteGroup",
                "qbusiness:UpdateUser",
                "qbusiness:ListGroups",
            ],
            "Resource": "*",
        },
    ],
}

In [ ]:
try:
    # Create the Box role
    box_role_response = iam.create_role(
        RoleName=box_role_name,
        AssumeRolePolicyDocument=json.dumps(box_trust_policy),
        Description="Role for Amazon Q Business Box",
    )
    print("Box role created successfully.")

    # Create and attach the custom inline policy
    iam.put_role_policy(
        RoleName=box_role_name,
        PolicyName="QBusinessBoxPermissions",
        PolicyDocument=json.dumps(box_policy_document),
    )
    print("Custom policy attached to Box role.")

    # Get the Box role ARN
    box_role_arn = box_role_response["Role"]["Arn"]
    print(f"Box Role ARN: {box_role_arn}")

except iam.exceptions.EntityAlreadyExistsException:
    print(f"Role '{box_role_name}' already exists.")
    box_role_response = iam.get_role(RoleName=box_role_name)
    box_role_arn = box_role_response["Role"]["Arn"]
    print(f"Box Role ARN: {box_role_arn}")

## Figure out if IDC exists in your account or create IDC

Let's see if we have an IDC instance. If so, get the ARN, if not create one. Sometimes the IDC for your account may be in another region (cross region IDC).

In [ ]:
# Define regions to check for IAM Identity Center
regions_to_check = [
    "us-east-1",
    "us-east-2",
    "us-west-1",
    "us-west-2",
    "eu-west-1",
    "eu-central-1",
    "ap-northeast-1",
    "ap-southeast-1",
]

In [ ]:
# Find IAM Identity Center instance across all regions
def find_idc_instance():
    # List of AWS regions to check
    print("Searching for IAM Identity Center instances across regions...")
    # Check each region for an Identity Center instance
    for check_region in regions_to_check:
        try:
            # Create SSO admin client for this region
            sso_admin = boto3.client(
                "sso-admin", region_name=check_region, verify=False
            )

            # List existing Identity Center instances
            response = sso_admin.list_instances()

            # Check if any instances exist in this region
            if response["Instances"]:
                instance_arn = response["Instances"][0]["InstanceArn"]
                instance_region = check_region
                print(
                    f"✅ Found IAM Identity Center instance in {check_region}: "
                    f"{instance_arn}"
                )
                return {
                    "identityType": "AWS_IAM_IDC",
                    "identityCenterInstanceArn": instance_arn,
                    "region": instance_region,
                }
        except Exception as e:
            # Continue to next region if there's an error
            continue

    # If no instance found, return None
    return None

In [ ]:
# Try to find existing IDC instance
idc_config = find_idc_instance()

# If no instance found, try to create one
if not idc_config:
    print(
        "No existing IAM Identity Center instances found. Attempting to "
        "create one..."
    )

    try:
        # Create Organizations client to check if organization exists
        org_client = boto3.client("organizations")
        org_client.describe_organization()

        # Create SSO admin client in current region
        sso_admin = boto3.client("sso-admin", region_name=region, verify=False)

        # Create Identity Center instance
        response = sso_admin.create_instance()
        print(f"✅ Created new IAM Identity Center instance in {region}")

        # Wait for instance to be available
        print("Waiting for instance to be available...")
        time.sleep(10)

        # Get the ARN of the new instance
        list_response = sso_admin.list_instances()
        if list_response["Instances"]:
            instance_arn = list_response["Instances"][0]["InstanceArn"]
            print(f"New instance ARN: {instance_arn}")
            idc_config = {
                "identityType": "AWS_IAM_IDC",
                "identityCenterInstanceArn": instance_arn,
                "region": region,
            }
    except Exception as e:
        print(f"Error creating IAM Identity Center instance: {str(e)}")

if not idc_config:
    print("❌ Could not find or create an IAM Identity Center instance")

## Create a user

For that IDC, let's create a user ("John Doe" or "john_doe@example.com").

In [ ]:
def create_idc_user(username, given_name, family_name):
    try:
        # Variables to store instance information
        instance_arn = None
        identity_store_id = None
        idc_region = None

        # Check each region for an Identity Center instance
        for check_region in regions_to_check:
            try:
                # Create SSO admin client for this region
                sso_admin = boto3.client(
                    "sso-admin", region_name=check_region, verify=False
                )

                # List existing Identity Center instances
                response = sso_admin.list_instances()

                # Check if any instances exist in this region
                if response.get("Instances"):
                    instance = response["Instances"][0]
                    instance_arn = instance["InstanceArn"]

                    # Get the Identity Store ID
                    instance_response = sso_admin.describe_instance(
                        InstanceArn=instance_arn
                    )
                    identity_store_id = instance_response["IdentityStoreId"]
                    idc_region = check_region
                    break
            except Exception:
                continue

        if not instance_arn or not identity_store_id:
            print("No IAM Identity Center instances found in any region")
            return None

        return instance_arn, identity_store_id, idc_region
    except Exception as e:
        print(f"Error finding Identity Center: {str(e)}")
        return None, None, None

In [ ]:
# Get Identity Center information
instance_arn, identity_store_id, idc_region = create_idc_user(
    email, first_name, family_name
)

if identity_store_id:
    try:
        # Create an Identity Store client in the correct region
        identity_store = boto3.client(
            "identitystore", region_name=idc_region, verify=False
        )

        # Create a new user with the provided username, given name, and family name
        user_response = identity_store.create_user(
            IdentityStoreId=identity_store_id,
            UserName=email,
            Name={"GivenName": first_name, "FamilyName": family_name},
            DisplayName=f"{first_name} {family_name}",
            Emails=[
                {
                    "Value": email if "@" in email else f"{email}@example.com",
                    "Type": "Work",
                    "Primary": True,
                }
            ],
        )

        user_id = user_response["UserId"]
        print(
            f"\nUser '{first_name} {family_name}' created successfully. id: "
            f"{user_id}, username: {email}"
        )
        print(
            "\nNote: For security reasons, "
            "password must be set manually through the AWS Console."
        )
    except Exception as e:
        print(f"Error creating user in IAM Identity Center: {str(e)}")
        user_id = None
else:
    print("Cannot create user without Identity Store ID")
    user_id = None

## Create Q Business Instance

We'll create a Q Business instance using the IAM role we just created.

In [ ]:
# Initialize the Q Business client with region
q_business = boto3.client("qbusiness", region_name=region, verify=False)

try:
    if idc_config:
        # Create the Q Business instance with IAM Identity Center
        create_params = {
            "displayName": instance_name,
            "roleArn": app_role_arn,
            "identityType": idc_config["identityType"],
            "identityCenterInstanceArn": idc_config["identityCenterInstanceArn"],
        }

        print(
            f"Creating Q Business application with IAM Identity Center from "
            f"{idc_config['region']}"
        )
        response = q_business.create_application(**create_params)

        print("Q Business instance creation initiated successfully!")
        print("\nResponse: ")
        print(json.dumps(response, indent=2))

        # Store the application ID for future reference
        application_id = response["applicationId"]
    else:
        print("Cannot create Q Business application without IAM Identity " "Center")

except Exception as e:
    print(f"Error creating Q Business instance: {str(e)}")

## Check Instance Status

Let's create a function to check the status of our Q Business instance

In [ ]:
def check_instance_status(application_id):
    try:
        response = q_business.get_application(applicationId=application_id)
        status = response["status"]
        print(f"Current status: {status}")
        return status
    except Exception as e:
        print(f"Error checking status: {str(e)}")
        return None

In [ ]:
# Check status periodically until the instance is ready
max_attempts = 30
attempt = 0

while attempt < max_attempts:
    status = check_instance_status(application_id)
    if status == "ACTIVE":
        print("\nQ Business instance is now active!")
        break
    elif status == "FAILED":
        print("\nQ Business instance creation failed!")
        break

    print(
        f"Waiting for instance to become active... (Attempt {attempt + 1}/{max_attempts})"
    )
    time.sleep(60)  # Wait for 60 seconds before checking again
    attempt += 1

## Add user to the application, and configure the subscription


In [ ]:
def add_user_with_pro_subscription(application_id, user_id):
    """
    Add a user to the Q Business instance with a Pro subscription.

    Parameters:
    - application_id: The ID of the Q Business application
    - user_id: The user ID from IAM Identity Center
    """
    try:
        # Initialize the Q Business client
        q_business = boto3.client("qbusiness", region_name=region, verify=False)

        # Check if user_id exists before creating subscription
        if user_id:
            try:
                subscription_response = q_business.create_subscription(
                    applicationId=application_id,
                    principal={"user": user_id},
                    type="Q_BUSINESS",
                )
                print(
                    f"User {user_id} added with subscription "
                    f"{subscription_response['subscriptionId']}"
                )
                return subscription_response
            except Exception as e:
                print(f"Error creating subscription: {str(e)}")
                return None
        else:
            print("User creation failed - user_id is None")
            return None
    except Exception as e:
        print(f"❌ Error adding user to Q Business: {str(e)}")
        return None

In [ ]:
# Add the user with Pro subscription
subscription_response = add_user_with_pro_subscription(application_id, user_id)

if subscription_response:
    print(f"✅ User with ID {user_id} added to Q Business with Pro subscription!")
    print(f"Q Business User ID: {user_id}")
    print(f"Subscription ID: {subscription_response['subscriptionId']}")
    print(json.dumps(subscription_response, indent=5))

## Print some debugging info

Just a checkpoint to print debugging info

In [ ]:
print(f"application_id: {application_id}")
print(f"user {user_id}")

print("list subscriptions: ")
print(json.dumps(q_business.list_subscriptions(applicationId=application_id), indent=5))
sub_id = q_business.list_subscriptions(applicationId=application_id)["subscriptions"][
    0
]["subscriptionId"]
print(f"subscription_id: {sub_id}")

## Create Index

Before creating a data source, we need to create an index that will store the crawled data.

In [ ]:
try:
    # Create the index with displayName and capacity configuration
    index_response = q_business.create_index(
        applicationId=application_id,
        displayName="box-index",
        description="Index for Box data source",
        type="ENTERPRISE",
        capacityConfiguration={
            "units": 1  # Specify the number of capacity units (1-10)
        },
    )

    print("Index creation initiated successfully!")
    print("\nResponse: ")
    print(json.dumps(index_response, indent=2))

    # Store the index ID for future reference
    index_id = index_response["indexId"]

except Exception as e:
    print(f"Error creating index: {str(e)}")

## Check Index Status

Let's monitor the status of our index creation.

In [ ]:
def check_index_status(application_id, index_id):
    try:
        response = q_business.get_index(applicationId=application_id, indexId=index_id)
        status = response["status"]
        print(f"Current status: {status}")
        return status
    except Exception as e:
        print(f"Error checking status: {str(e)}")
        return None

In [ ]:
# Check status periodically until the index is ready
max_attempts = 30
attempt = 0

while attempt < max_attempts:
    status = check_index_status(application_id, index_id)
    if status == "ACTIVE":
        print("\nIndex is now active!")
        break
    elif status == "FAILED":
        print("\nIndex creation failed!")
        break

    print(
        f"Waiting for index to become active... (Attempt {attempt + 1}/{max_attempts})"
    )
    time.sleep(60)  # Wait for 60 seconds before checking again
    attempt += 1

## Create retriever

Along with the index, you'll need a retriever to use the index.

In [ ]:
try:
    # Create a retriever with correct configuration
    retriever_response = q_business.create_retriever(
        applicationId=application_id,
        displayName="box-retriever",
        type="NATIVE_INDEX",
        configuration={
            "nativeIndexConfiguration": {
                "indexId": index_id,
            }
        },
        roleArn=app_role_arn,
    )

    print("Retriever creation initiated successfully!")
    print("\nResponse: ")
    print(json.dumps(retriever_response, indent=2))

    # Store the retriever ID for future reference
    retriever_id = retriever_response["retrieverId"]

except Exception as e:
    print(f"Error creating retriever: {str(e)}")

## Configure Box Data Source

Now that our Q Business instance and index are active, let's configure a Box data source.

In [ ]:
# Create Box configuration
box_config = {
    "ingestionMode": "SCHEDULED",
    "connectionConfiguration": {
        "repositoryEndpointMetadata": {"enterpriseId": box_enterprise_id}
    },
    "enableIdentityCrawler": True,
    "secretArn": f"arn:aws:secretsmanager:{region}:{identity['Account']}:secret:{box_secret_name}",
    "syncMode": "FORCED_FULL_CRAWL",
    "additionalProperties": {
        "inclusionPatterns": [],
        "includeSupportedFileType": False,
        "crawlWebLinks": False,
        "crawlTasks": False,
        "crawlComments": False,
        "fieldForUserId": "uuid",
        "maxFileSizeInMegaBytes": "50",
        "enableDeletionProtection": False,
        "folderIDs": [],
        "isCrawlAcl": True,
        "exclusionPatterns": [],
        "deletionProtectionThreshold": "0",
    },
    "type": "BOX",
    "version": "1.0.0",
}

In [ ]:
# Add field mappings to the configuration
box_config["repositoryConfigurations"] = {
    "file": {
        "fieldMappings": [
            {
                "dataSourceFieldName": "bx_uri",
                "indexFieldName": "_source_uri",
                "indexFieldType": "STRING",
            },
            {
                "dataSourceFieldName": "bx_category",
                "indexFieldName": "_category",
                "indexFieldType": "STRING",
            },
        ]
    }
}

In [ ]:
try:
    # Create the data source
    response = q_business.create_data_source(
        applicationId=application_id,
        indexId=index_id,
        displayName="box-source",
        configuration=box_config,
        roleArn=box_role_arn,
    )

    data_source_id = response["dataSourceId"]
    print(f"Box data source created: {data_source_id}")

except Exception as e:
    print(f"Error: {str(e)}")

## Check Data Source Status

Let's monitor the status of our Box data source.

In [ ]:
def check_data_source_status(application_id, index_id, data_source_id):
    try:
        response = q_business.get_data_source(
            applicationId=application_id, indexId=index_id, dataSourceId=data_source_id
        )
        status = response["status"]
        print(f"Current status: {status}")
        return status
    except Exception as e:
        print(f"Error checking status: {str(e)}")
        return None

In [ ]:
# Check status periodically until the data source is ready
max_attempts = 30
attempt = 0

while attempt < max_attempts:
    status = check_data_source_status(application_id, index_id, data_source_id)
    if status == "ACTIVE":
        print("\nBox data source is now active!")
        break
    elif status == "FAILED":
        print("\nBox data source creation failed!")
        break

    print(
        f"Waiting for data source to become active... (Attempt {attempt + 1}/{max_attempts})"
    )
    time.sleep(60)  # Wait for 60 seconds before checking again
    attempt += 1

## Start a Data Source Sync

This cell initiates a sync job for the Box data source

In [ ]:
try:
    # Start a sync job for the data source
    sync_response = q_business.start_data_source_sync_job(
        applicationId=application_id, indexId=index_id, dataSourceId=data_source_id
    )

    if sync_response["ResponseMetadata"]["HTTPStatusCode"] == 200:
        print("Data source sync job initiated successfully!")
        print("\nResponse: ")
        print(json.dumps(sync_response, indent=2))

        # Store the execution ID for future reference
        execution_id = sync_response["executionId"]
    else:
        print(
            f"Error starting sync job. Status code: "
            f"{sync_response['ResponseMetadata']['HTTPStatusCode']}"
        )
        print("\nResponse: ")
        print(json.dumps(sync_response, indent=2))

except Exception as e:
    print(f"Error starting data source sync job: {str(e)}")

## Check Sync Job Status

Monitor the status of the data source sync job

In [ ]:
def check_sync_job_status(application_id, index_id, data_source_id):
    try:
        response = q_business.list_data_source_sync_jobs(
            applicationId=application_id, indexId=index_id, dataSourceId=data_source_id
        )

        # Get the most recent sync job from history
        if response["history"]:
            latest_job = response["history"][0]  # Jobs are returned in descending order
            status = latest_job["status"]
            print(f"Current sync job status: {status}")

            # Print metrics if available
            if "metrics" in latest_job:
                print("\nSync metrics: ")
                print(json.dumps(latest_job["metrics"], indent=2))

            # Print error if job failed
            if status == "FAILED" and "error" in latest_job:
                print("\nError details: ")
                print(json.dumps(latest_job["error"], indent=2))

            return status
        else:
            print("No sync jobs found in history")
            return None

    except Exception as e:
        print(f"Error checking sync job status: {str(e)}")
        return None

In [ ]:
# Check status periodically until the sync job is complete
max_attempts = 30
attempt = 0

while attempt < max_attempts:
    status = check_sync_job_status(application_id, index_id, data_source_id)
    if status == "SUCCEEDED":
        print("\nSync job completed successfully!")
        break
    elif status in ["FAILED", "ABORTED"]:
        print("\nSync job failed!")
        break
    elif status in ["SYNCING", "SYNCING_INDEXING"]:
        print(
            f"Waiting for sync job to complete... (Attempt {attempt + 1}/{max_attempts})"
        )
        time.sleep(60)  # Wait for 60 seconds before checking again
        attempt += 1
    else:
        print(f"\nUnexpected status: {status}")
        break

## Clean Up (Optional)

If you want to delete the Q Business instance, data source, index, and the IAM role, you can use the following code: 

In [ ]:
# Run this cell to delete all resources created in this notebook
try:
    # Delete the retriever
    print("Deleting retriever...")
    q_business.delete_retriever(applicationId=application_id, retrieverId=retriever_id)
    print("Retriever deletion initiated successfully!")

    # Delete the data source
    print("Deleting data source...")
    q_business.delete_data_source(
        applicationId=application_id, indexId=index_id, dataSourceId=data_source_id
    )
    print("Data source deletion initiated successfully!")

    # Delete the index
    print("Deleting index...")
    q_business.delete_index(applicationId=application_id, indexId=index_id)
    print("Index deletion initiated successfully!")

    # Delete the subscription
    print("Deleting subscription...")
    q_business.delete_subscription(applicationId=application_id, subscriptionId=sub_id)
    print("Subscription deletion initiated successfully!")

    # Delete the Q Business instance
    print("Deleting Q Business application...")
    q_business.delete_application(applicationId=application_id)
    print("Q Business instance deletion initiated successfully!")

    # Delete the Box IAM role
    print("Deleting Box IAM role...")
    # First delete the inline policy
    iam.delete_role_policy(RoleName=box_role_name, PolicyName="QBusinessBoxPermissions")
    print(f"Inline policy deleted from {box_role_name}")

    # Then delete the role
    iam.delete_role(RoleName=box_role_name)
    print(f"Role {box_role_name} deleted")

    # Delete the Box secret
    print("Deleting Box secret...")
    secrets_client.delete_secret(
        SecretId=box_secret_name, ForceDeleteWithoutRecovery=True
    )
    print(f"Secret {box_secret_name} deleted")

    print("\n✅ All resources have been cleaned up successfully!")
except Exception as e:
    print(f"Error during cleanup: {str(e)}")